In [20]:
import json
import pandas as pd
import numpy as np
from pymongo import MongoClient
from collections import Counter
%matplotlib inline

Get freelancer profile by key

Endpoint
GET /api/profiles/v1/providers/{profile_key}.{format}

In [21]:
def json_prep(in_file):
    # read the entire file into a python array
    with open(in_file, 'rb') as f:
        data = f.readlines()

    # remove the trailing "\n" from each line
    data = map(lambda x: x.rstrip(), data)

    # each element of 'data' is an individual JSON object.
    # i want to convert it into an *array* of JSON objects
    # which, in and of itself, is one large JSON object
    # basically... add square brackets to the beginning
    # and end, and have all the individual business JSON objects
    # separated by a comma
    data_json_str = "[" + ','.join(data) + "]"

    data_list_of_dicts = json.loads(data_json_str)
    
#     now, load it into pandas
#     out_df = pd.read_json(data_json_str)
    return data_list_of_dicts

## Make lists of dicts out of JSON data

In [5]:
as_profiles_lod = json_prep('../data/profiles_by_skill/apache-spark.txt')

In [6]:
print len(as_profiles_lod), type(as_profiles_lod)  # it's a list of dicts

655 <type 'list'>


In [7]:
ids = []
for dict_item in as_profiles_lod:
    ids.append(dict_item['id'])

In [8]:
print len(ids)
print ids[:5]

655
[u'~019fb185473d0c4789', u'~01db54f572948cbc45', u'~01b1e3d1a582370c49', u'~01a33ea6f9ce09ac24', u'~01a644f4e0579a50de']


## Insert into MongoDB

In [18]:
client = MongoClient()
# Access/Initiate Database
db = client['upwork_db']

In [42]:
db.collection_names()

[]

In [43]:
# Access/Initiate Table
profiles = db['apache-spark_profiles']

In [44]:
profiles.insert_many(as_profiles_lod)

In [45]:
db.collection_names()

[u'apache-spark_profiles']

In [47]:
db[u'apache-spark_profiles'].count()

655

In [46]:
db[u'apache-spark_profiles'].find_one()

{u'_id': ObjectId('571c223516d5977e5f226c39'),
 u'categories2': [u'IT & Networking'],
 u'country': u'Canada',
 u'description': u'10+ years experience as an Database (MySQL,Oracle,MS SQL) Administrator\n\n5+ years experience as an Linux, Ejabberd, Apache, Nginx,FreePBX,LVS, HAProxy administrator \n\n3+ years experience as an NoSQL(MongoDB,TokuMx), NewSQL(Cassandra, Impala),Data Warehouse (Infobright, Vertica), Elasticsearch, Logstash Administrator \n\nExcellent skills in implementing replication, allocate storage, upgrade, backup and recovery and maintain database access control and system security \n\nRecover corrupted table, dropped table without backup.\n\nWorked on databases of more than 10K QPS and data increased by TB each month. \n\nExpert in query tuning, schema and server optimization \n\nExperience in Percona toolkit, XtraDB cluster, different replication typologies with Tungsten replicator\n\nProven problem-solving ability in Linux administration and performance tuning\n\n\nE

In [27]:
# Deletes collection (table)
db['test_table'].drop()

In [28]:
db.collection_names()

[]

In [26]:
list(tab.find())

[]

In [17]:
client.close()

In [51]:
fl = ['ajax.txt', 'angularjs.txt', 'apache-hive.txt', 'apache-kafka.txt', 'apache-spark.txt', 'big-data.txt', 'bootstrap.txt', 'c#.txt', 'cassandra.txt', 'css.txt', 'css3.txt', 'data-analysis.txt', 'data-mining.txt', 'data-modeling.txt', 'data-science.txt', 'data-visualization.txt', 'django-framework.txt', 'hadoop.txt', 'hbase.txt', 'html.txt', 'html5.txt', 'ibm-spss.txt', 'java.txt', 'javascript.txt', 'jquery.txt', 'laravel-framework.txt', 'machine-learning.txt', 'mapreduce.txt', 'matlab.txt', 'mongodb.txt', 'mysql.txt', 'node.js.txt', 'nosql.txt', 'php.txt', 'pig.txt', 'postgresql.txt', 'predictive-analytics.txt', 'python-numpy.txt', 'python-scipy.txt', 'python.txt', 'r.txt', 'ruby-on-rails.txt', 'ruby.txt', 'sas.txt', 'scala.txt', 'spring-framework.txt', 'sql.txt', 'statistics.txt', 'tableau.txt', 'twitter-bootstrap.txt', 'wordpress.txt']

In [22]:
import os

In [24]:
project_directory_path = '/Users/JerryTsai/userjst/individ/knowledg/cur/galvanize/project-upwork'
# data_directory_path = os.getcwd() + '/data/profiles_by_skill'
list_of_filenames = os.listdir(project_directory_path + '/data/profiles_by_skill')
print list_of_filenames

['.DS_Store', 'apache-hive.txt', 'apache-kafka.txt', 'apache-spark.txt', 'big-data.txt', 'cassandra.txt', 'data-analysis.txt', 'data-mining.txt', 'data-modeling.txt', 'data-science.txt', 'data-visualization.txt', 'full_stack', 'hadoop.txt', 'hbase.txt', 'ibm-spss.txt', 'java.txt', 'machine-learning.txt', 'mapreduce.txt', 'matlab.txt', 'mongodb.txt', 'mysql.txt', 'nosql.txt', 'pig.txt', 'predictive-analytics.txt', 'python-numpy.txt', 'python-scipy.txt', 'python.txt', 'r.txt', 'sas.txt', 'scala.txt', 'spring-framework.txt', 'sql.txt', 'statistics.txt', 'tableau.txt']


In [ ]:
all_ids_set = set()
for filename in list_of_filenames:
    # Set up 
    if filename[-4:] == '.txt':
        skill_name = filename.split('.')[0].replace('-', '_').replace('#', 'Sharp')
    filepath = project_directory_path + '/data/profiles_by_skill/' + filename
    
    profiles_lod = json_prep(filepath)
    ids_list = []
    for profile_as_dict in profiles_lod:
        ids_list.append(profile_as_dict['id'])
    
    ids_set = set(ids_list)
    print 'Skill: {}, ID count {}'.format(skill_name, len(ids_set))
    all_ids_set.update(ids_set)
    
print 'All ID counts: {}'.format(len(all_ids_set))

# Do NOT run Below unless you make all_ids_set small

In [19]:
list_size = 5
counter = 0
set_size_m_1 = len(all_ids_set) - 1
for index, id in enumerate(all_ids_set):
    remainder = counter % list_size
    if remainder == 0:
        id_list = []
    id_list.append(id)
    if remainder == (list_size-1) or counter == set_size_m_1:
        profiles_str = ';'.join(id_list)
        print profiles_str
    counter += 1

aa;ii;jj;bb;cc
dd;m;l;gg;ee
hh;kk;ff;nn


In [ ]:
The freelancer's profile key or a list of keys, separated by semicolon (';'). 
The number of keys per request is limited to 20.